In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot 
init_notebook_mode(connected = True)

In [ ]:
data = pd.read_csv('../input/jpx-tokyo-stock-exchange-prediction/train_files/stock_prices.csv') 
stock_list=pd.read_csv('../input/jpx-tokyo-stock-exchange-prediction/stock_list.csv') 

In [ ]:
data.head(3)

### Basic data quality checks

In [ ]:
data.isna().sum()

In [ ]:
print("Total Null Target Rows = " ,data["Target"].isnull().sum())
print("Percentage of NUll Target rows in Training Data = {:.2f}%".format(data["Target"].isnull().sum()*100 / data.shape[0] ))


### Data distribution by Sectors

In [ ]:
data_distr=data.groupby('SecuritiesCode').size().reset_index(name='total')
stock_list_data=pd.merge(stock_list,data_distr, how='left',on=['SecuritiesCode'])
stock_list_data=stock_list_data.groupby(['33SectorName']).total.sum().reset_index(name='total')
fig = px.bar(x = stock_list_data["33SectorName"],
             y = stock_list_data['total'] , 
             color = stock_list_data["33SectorName"] ,
             color_continuous_scale="Emrld") 
fig.update_xaxes(title="Assets")
fig.update_yaxes(title = "Number of Rows")
fig.update_layout(showlegend = True,
    title = {
        'text': 'Data Distribution ',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'} ,
        template="plotly_white")
fig.show()

Most of the stock exchanges listed belong to Information and Communication and Services

### CandleStick Charts to understand Market Performance

In [ ]:
def get_data(stock_id ,data= data ):
    df = data[data["SecuritiesCode"]==stock_id].reset_index(drop = True)
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.set_index('Date')
    return df
nippon = get_data(stock_id = 1332)
sakatha=get_data(stock_id = 1377)

In [ ]:
def candelstick_chart(data,title):
    candlestick = go.Figure(data = [go.Candlestick(x =data.index, 
                                               open = data[('Open')], 
                                               high = data[('High')], 
                                               low = data[('Low')], 
                                               close = data[('Close')])])
    candlestick.update_xaxes(title_text = 'Time',
                             rangeslider_visible = True)

    candlestick.update_layout(
    title = {
        'text': '{:} Candelstick Chart'.format(title),
        'y':0.90,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'} , 
    template="plotly_white")

    candlestick.update_yaxes(title_text = 'Price in USD', ticksuffix = '$')
    return candlestick

In [ ]:
nipponplot = candelstick_chart(nippon,title = "Nippon")
nipponplot.show()

In [ ]:
sakathaplot = candelstick_chart(sakatha,title = "Sakata Seed Corporation")
sakathaplot.show()

### Area/Volume Charts

In [ ]:
def vol_traded(data ,title,color):
    area = px.area(data_frame=data,
               x = data.index ,
               y = "Volume",
               markers = True)
    area.update_traces(line_color=color)
    area.update_xaxes(
        title_text = 'Time',
        rangeslider_visible = True)
    area.update_yaxes(title_text = 'Number of trades every minute')
    area.update_layout(showlegend = True,
        title = {
            'text': '{:} Volume Traded'.format(title),
            'y':0.94,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
        template="plotly_white")
    return area

In [ ]:
vol_traded(sakatha[-50:], "Sakatha Seed Corporation For the last 50 days",color = "Red")

In [ ]:
vol_traded(nippon, "Nippon",color = "Red")

### Understanding Stock Volatility

In [ ]:
nippon["Return"] = (nippon["Close"]/nippon["Close"].shift(1)) - 1

In [ ]:
fig = px.histogram(nippon, x = "Return")

fig.update_layout(
    plot_bgcolor = "white",
    title_text="Stock Volatility of Nippon"
)

fig.update_yaxes(showticklabels = True, showline = True, linewidth = 2, linecolor = "black",
                title_text="<b>Value</b>")
fig.update_xaxes(showticklabels = True, showline = True, linewidth = 2, linecolor = "black", 
                 title_text="<b>Return</b>")
iplot(fig)

#### This is a WIP please upvote if you find the notebook useful